In [5]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus

# Load DAM data
dat1 = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/DAM_QRA/QR/rf_Q_DAM_1-12.csv")
dat1 = pd.DataFrame(dat1)

quantiles = [10, 30, 50, 70, 90]
Q_DAM = {}
for q in quantiles:
    column_names = [f"EURPrices+{i}_Forecast_{q}" for i in range(24)]
    Q_DAM[q] = dat1[column_names].dropna().stack().reset_index()
    Q_DAM[q]["Price"] = Q_DAM[q].iloc[:, 2]
    Q_DAM[q] = Q_DAM[q].reindex(Q_DAM[q].index.repeat(2)).reset_index(drop=True)

column_names_dam = [f"EURPrices+{i}" for i in range(24)]
Y_r_DAM = dat1[column_names_dam].dropna().stack().reset_index()
Y_r_DAM = Y_r_DAM.reindex(Y_r_DAM.index.repeat(2)).reset_index(drop=True)
Y_r_DAM["Price"] = Y_r_DAM.iloc[:, 2]

# Load BM data
dat2 = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/BM_QRA/QR/rf_Q_1-12.csv")
dat2 = pd.DataFrame(dat2)

quantiles = [10, 30, 50, 70, 90]
Q_BM = {}
for q in quantiles:
    column_names = [f"lag_{i}y_Forecast_{q}" for i in range(2, 18)]
    Q_BM[q] = dat2[column_names].dropna().stack().reset_index()
    Q_BM[q]["Price"] = Q_BM[q].iloc[:, 2]

column_names_bm = [f"lag_{i}y" for i in range(2, 18)]
Y_r_BM = dat2[column_names_bm].dropna().stack().reset_index()
Y_r_BM["Price"] = Y_r_BM.iloc[:, 2]

# Set battery parameters
battery_efficiency_charge = 0.80
battery_efficiency_discharge = 0.98
Total_Daily_Volume = 6
max_battery_capacity = 1
min_battery_capacity = 0
ramp_rate_charge = 1
ramp_rate_discharge = 1

# Define time periods (48 half-hour periods)
time_periods = range(0, len(Y_r_DAM), 48)

# Define quantile pairs
quantile_pairs = [(50, 50)]

# Initialize dictionaries to store total profits for each quantile pair
total_profits = {pair: 0 for pair in quantile_pairs}

# Loop over each quantile pair
for alpha_dam, alpha_bm in quantile_pairs:
    results = []
    current_charge = min_battery_capacity

    # Loop over each time period (day)
    for start_idx in time_periods:
        end_idx = start_idx + 48

        if end_idx > len(Y_r_DAM):
            break

        print(f"Initial charge: {current_charge}")

        # Create a linear programming problem
        problem = LpProblem("Dual_Market_Trading_Strategy", LpMaximize)

        # Define decision variables for DAM and BM
        buy_action_dam = LpVariable.dicts("Buy_Action_DAM", range(48), cat='Binary')
        sell_action_dam = LpVariable.dicts("Sell_Action_DAM", range(48), cat='Binary')
        buy_dam = LpVariable.dicts("Buy_DAM", range(48), lowBound=0, cat='Continuous')
        sell_dam = LpVariable.dicts("Sell_DAM", range(48), lowBound=0, cat='Continuous')
        charge_dam = LpVariable.dicts("Charge_DAM", range(49), lowBound=min_battery_capacity, upBound=max_battery_capacity, cat='Continuous')

        buy_action_bm = LpVariable.dicts("Buy_Action_BM", range(48), cat='Binary')
        sell_action_bm = LpVariable.dicts("Sell_Action_BM", range(48), cat='Binary')
        buy_bm = LpVariable.dicts("Buy_BM", range(48), lowBound=0, cat='Continuous')
        sell_bm = LpVariable.dicts("Sell_BM", range(48), lowBound=0, cat='Continuous')
        charge_bm = LpVariable.dicts("Charge_BM", range(49), lowBound=min_battery_capacity, upBound=max_battery_capacity, cat='Continuous')

        # Define the objective function
        dam_profit = lpSum([battery_efficiency_discharge * Q_DAM[alpha_dam]["Price"].iloc[t] * sell_dam[t] - 
                            (Q_DAM[alpha_dam]["Price"].iloc[t] / battery_efficiency_charge) * buy_dam[t] for t in range(48)])
        bm_profit = lpSum([battery_efficiency_discharge * Q_BM[alpha_bm]["Price"].iloc[t] * sell_bm[t] - 
                           (Q_BM[alpha_bm]["Price"].iloc[t] / battery_efficiency_charge) * buy_bm[t] for t in range(48)])
        problem += dam_profit + bm_profit

        # DAM constraints
        # Initial battery charge constraint
        problem += charge_dam[0] == current_charge, "Initial_Charge_DAM"

        # Define the total daily volume constraint for DAM
        problem += lpSum([buy_dam[t] for t in range(48)]) + lpSum([sell_dam[t] for t in range(48)]) == Total_Daily_Volume, "Total_Daily_Volume_DAM"

        # Ensure that each hour has at most one buy or sell action
        for t in range(48):
            problem += buy_action_dam[t] + sell_action_dam[t] <= 1, f"One_Action_Per_Hour_DAM_{t}"

        # Link action variables to buy and sell variables for DAM
        for t in range(48):
            problem += buy_dam[t] <= ramp_rate_charge * buy_action_dam[t], f"Buy_Action_Link_DAM_{t}"
            problem += sell_dam[t] <= ramp_rate_discharge * sell_action_dam[t], f"Sell_Action_Link_DAM_{t}"

        # Battery charge constraints for DAM
        for t in range(48):
            problem += charge_dam[t + 1] == charge_dam[t] + buy_dam[t] - sell_dam[t], f"Charge_Update_DAM_{t}"
            problem += charge_dam[t] <= max_battery_capacity, f"Max_Capacity_DAM_{t}"
            problem += charge_dam[t] >= min_battery_capacity, f"Min_Capacity_DAM_{t}"

            # Charging constraints for DAM
            problem += buy_dam[t] <= max_battery_capacity - charge_dam[t], f"Charging_Constraint_DAM_{t}"
            problem += buy_dam[t] <= ramp_rate_charge, f"Charging_Ramp_Rate_Constraint_DAM_{t}"

            # Discharging constraints for DAM
            problem += sell_dam[t] <= charge_dam[t] - min_battery_capacity, f"Discharging_Constraint_DAM_{t}"
            problem += sell_dam[t] <= ramp_rate_discharge, f"Discharging_Ramp_Rate_Constraint_DAM_{t}"

        # BM constraints
        # Initial battery charge constraint for BM
        problem += charge_bm[0] == charge_dam[48], "Initial_Charge_BM"

        # Define the total daily volume constraint for BM
        problem += lpSum([buy_bm[t] for t in range(48)]) <= Total_Daily_Volume, "Total_Daily_Volume_Buy_BM"
        problem += lpSum([sell_bm[t] for t in range(48)]) <= Total_Daily_Volume, "Total_Daily_Volume_Sell_BM"

        # Ensure that each hour has at most one buy or sell action for BM
        for t in range(48):
            problem += buy_action_bm[t] + sell_action_bm[t] <= 1, f"One_Action_Per_Hour_BM_{t}"

        # Link action variables to buy and sell variables for BM
        for t in range(48):
            problem += buy_bm[t] <= ramp_rate_charge * buy_action_bm[t], f"Buy_Action_Link_BM_{t}"
            problem += sell_bm[t] <= ramp_rate_discharge * sell_action_bm[t], f"Sell_Action_Link_BM_{t}"

        # Battery charge constraints for BM
        for t in range(48):
            problem += charge_bm[t + 1] == charge_bm[t] + buy_bm[t] - sell_bm[t], f"Charge_Update_BM_{t}"
            problem += charge_bm[t] <= max_battery_capacity, f"Max_Capacity_BM_{t}"
            problem += charge_bm[t] >= min_battery_capacity, f"Min_Capacity_BM_{t}"
            
        # Charging constraints for BM
        for t in range(48):
            problem += buy_bm[t] <= max_battery_capacity - charge_bm[t], f"Charging_Constraint_BM_{t}"
            problem += buy_bm[t] <= ramp_rate_charge, f"Charging_Ramp_Rate_Constraint_BM_{t}"

            # Discharging constraints for BM
            problem += sell_bm[t] <= charge_bm[t] - min_battery_capacity, f"Discharging_Constraint_BM_{t}"
            problem += sell_bm[t] <= ramp_rate_discharge, f"Discharging_Ramp_Rate_Constraint_BM_{t}"

        # Objective function for BM (already defined above with dam_profit + bm_profit)

        # Solve the problem
        problem.solve()

        # Check if the solution is optimal
        if LpStatus[problem.status] == 'Optimal':
            # Extract buy and sell times and amounts for DAM
            buy_times_dam = [t for t in range(48) if buy_dam[t].varValue > 0]
            sell_times_dam = [t for t in range(48) if sell_dam[t].varValue > 0]

            # Extract buy and sell times and amounts for BM
            buy_times_bm = [t for t in range(48) if buy_bm[t].varValue > 0]
            sell_times_bm = [t for t in range(48) if sell_bm[t].varValue > 0]

            # Ensure unique buy-sell pairs for DAM
            used_hours_dam = set()
            valid_trades_dam = []
            for bt in buy_times_dam:
                if bt in used_hours_dam:
                    continue
                for st in sell_times_dam:
                    if st in used_hours_dam or bt >= st:
                        continue
                    # Calculate expected profit for DAM
                    buy_price_dam = Q_DAM[alpha_dam]["Price"].iloc[start_idx + bt]
                    sell_price_dam = Q_DAM[alpha_dam]["Price"].iloc[start_idx + st]
                    expected_profit_dam = (battery_efficiency_discharge * sell_price_dam * sell_dam[st].varValue) - ((buy_price_dam * buy_dam[bt].varValue) / battery_efficiency_charge)

                    # Only consider the trade if expected profit is greater than 0 for DAM
                    if expected_profit_dam > 0:
                        valid_trades_dam.append((bt, st))
                        used_hours_dam.add(bt)
                        used_hours_dam.add(st)
                        break  # Only one valid sell per buy for DAM

            # Ensure unique buy-sell pairs for BM
            used_hours_bm = set()
            valid_trades_bm = []
            for bt in buy_times_bm:
                if bt in used_hours_bm:
                    continue
                for st in sell_times_bm:
                    if st in used_hours_bm or bt >= st:
                        continue
                    # Calculate expected profit for BM
                    buy_price_bm = Q_BM[alpha_bm]["Price"].iloc[start_idx + bt]
                    sell_price_bm = Q_BM[alpha_bm]["Price"].iloc[start_idx + st]
                    expected_profit_bm = (battery_efficiency_discharge * sell_price_bm * sell_bm[st].varValue) - ((buy_price_bm * buy_bm[bt].varValue) / battery_efficiency_charge)

                    # Only consider the trade if expected profit is greater than 0 for BM
                    if expected_profit_bm > 0:
                        valid_trades_bm.append((bt, st))
                        used_hours_bm.add(bt)
                        used_hours_bm.add(st)
                        break  # Only one valid sell per buy for BM

            # Calculate profit for DAM and BM
            for bt_dam, st_dam in valid_trades_dam:
                # Calculate profit using real prices for DAM
                buy_price_dam = Y_r_DAM.iloc[start_idx + bt_dam]["Price"]
                sell_price_dam = Y_r_DAM.iloc[start_idx + st_dam]["Price"]
                buy_amount_dam = buy_dam[bt_dam].varValue
                sell_amount_dam = sell_dam[st_dam].varValue
                profit_dam = (battery_efficiency_discharge * sell_price_dam * sell_amount_dam) - ((buy_price_dam * buy_amount_dam) / battery_efficiency_charge)
                results.append(profit_dam)
                print(f"Day {start_idx // 48 + 1}, Alpha DAM {alpha_dam}: Buy at {bt_dam} ({buy_amount_dam} MW), Sell at {st_dam} ({sell_amount_dam} MW), Profit: {profit_dam}")

            for bt_bm, st_bm in valid_trades_bm:
                # Calculate profit using real prices for BM
                buy_price_bm = Y_r_BM.iloc[start_idx + bt_bm]["Price"]
                sell_price_bm = Y_r_BM.iloc[start_idx + st_bm]["Price"]
                buy_amount_bm = buy_bm[bt_bm].varValue
                sell_amount_bm = sell_bm[st_bm].varValue
                profit_bm = (battery_efficiency_discharge * sell_price_bm * sell_amount_bm) - ((buy_price_bm * buy_amount_bm) / battery_efficiency_charge)
                results.append(profit_bm)
                print(f"Day {start_idx // 48 + 1}, Alpha BM {alpha_bm}: Buy at {bt_bm} ({buy_amount_bm} MW), Sell at {st_bm} ({sell_amount_bm} MW), Profit: {profit_bm}")

            # Update the current charge for the next day
            current_charge = charge_bm[48].varValue

    # Calculate the sum of profits for the current quantile pair
    total_profit = sum(results)
    total_profits[(alpha_dam, alpha_bm)] = total_profit
    print(f"Total Profit for Alpha {alpha_dam}-{alpha_bm}: {total_profit}\n")

# Print the total profits for all quantile pairs
for pair, profit in total_profits.items():
    print(f"Total Profit for Alpha {pair[0]}-{pair[1]}: {profit}")




Initial charge: 0
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ciaran/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/4e2d4f5b59cd4b8ab76af23473447f8a-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/4e2d4f5b59cd4b8ab76af23473447f8a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 970 COLUMNS
At line 3470 RHS
At line 4436 BOUNDS
At line 4727 ENDATA
Problem MODEL has 965 rows, 482 columns and 1923 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 18.5249 - 0.00 seconds
Cgl0004I processed model has 573 rows, 475 columns (190 integer (190 of which binary)) and 1520 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -18.5249
Cbc0038I Relaxing continuous gives -18.5249
Cbc0038I Before mini branch and bound, 190 integers at bound fixed and 285 

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ciaran/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/b38e255b7dd244da934966add2f493c5-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/b38e255b7dd244da934966add2f493c5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 970 COLUMNS
At line 3470 RHS
At line 4436 BOUNDS
At line 4727 ENDATA
Problem MODEL has 965 rows, 482 columns and 1923 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 18.5249 - 0.00 seconds
Cgl0004I processed model has 573 rows, 475 columns (190 integer (190 of which binary)) and 1520 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -18.5249
Cbc0038I Relaxing continuous gives -18.5249
Cbc0038I Before mini branch and bound, 190 integers at bound fixed and 285 continuous
Cbc0038

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ciaran/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/df86a15fd7e243dcb76e2fbac72a8d3d-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/df86a15fd7e243dcb76e2fbac72a8d3d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 970 COLUMNS
At line 3470 RHS
At line 4436 BOUNDS
At line 4727 ENDATA
Problem MODEL has 965 rows, 482 columns and 1923 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 18.5249 - 0.00 seconds
Cgl0004I processed model has 573 rows, 475 columns (190 integer (190 of which binary)) and 1520 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -18.5249
Cbc0038I Relaxing continuous gives -18.5249
Cbc0038I Before mini branch and bound, 190 integers at bound fixed and 285 continuous
Cbc0038

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ciaran/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/69818ecfa8d442a78ff42bc8cf9b8cc4-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/69818ecfa8d442a78ff42bc8cf9b8cc4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 970 COLUMNS
At line 3470 RHS
At line 4436 BOUNDS
At line 4727 ENDATA
Problem MODEL has 965 rows, 482 columns and 1923 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 18.5249 - 0.00 seconds
Cgl0004I processed model has 573 rows, 475 columns (190 integer (190 of which binary)) and 1520 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -18.5249
Cbc0038I Relaxing continuous gives -18.5249
Cbc0038I Before mini branch and bound, 190 integers at bound fixed and 285 continuous
Cbc0038

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ciaran/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/792abdceed4e4ee0bc498dbd7ec221bc-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/792abdceed4e4ee0bc498dbd7ec221bc-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 970 COLUMNS
At line 3470 RHS
At line 4436 BOUNDS
At line 4727 ENDATA
Problem MODEL has 965 rows, 482 columns and 1923 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 18.5249 - 0.00 seconds
Cgl0004I processed model has 573 rows, 475 columns (190 integer (190 of which binary)) and 1520 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -18.5249
Cbc0038I Relaxing continuous gives -18.5249
Cbc0038I Before mini branch and bound, 190 integers at bound fixed and 285 continuous
Cbc0038

...

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ciaran/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/a37b10a64c274b1b920c4349fd149b62-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/a37b10a64c274b1b920c4349fd149b62-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 970 COLUMNS
At line 3470 RHS
At line 4436 BOUNDS
At line 4727 ENDATA
Problem MODEL has 965 rows, 482 columns and 1923 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 18.5249 - 0.00 seconds
Cgl0004I processed model has 573 rows, 475 columns (190 integer (190 of which binary)) and 1520 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -18.5249
Cbc0038I Relaxing continuous gives -18.5249
Cbc0038I Before mini branch and bound, 190 integers at bound fixed and 285 continuous
Cbc0038

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ciaran/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/5c47224277c9441f9a513ec4d523872a-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/5c47224277c9441f9a513ec4d523872a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 970 COLUMNS
At line 3470 RHS
At line 4436 BOUNDS
At line 4727 ENDATA
Problem MODEL has 965 rows, 482 columns and 1923 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 18.5249 - 0.00 seconds
Cgl0004I processed model has 573 rows, 475 columns (190 integer (190 of which binary)) and 1520 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -18.5249
Cbc0038I Relaxing continuous gives -18.5249
Cbc0038I Before mini branch and bound, 190 integers at bound fixed and 285 continuous
Cbc0038

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ciaran/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/903c2571acd3427387bb5d5c9e8ae54b-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/903c2571acd3427387bb5d5c9e8ae54b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 970 COLUMNS
At line 3470 RHS
At line 4436 BOUNDS
At line 4727 ENDATA
Problem MODEL has 965 rows, 482 columns and 1923 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 18.5249 - 0.00 seconds
Cgl0004I processed model has 573 rows, 475 columns (190 integer (190 of which binary)) and 1520 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -18.5249
Cbc0038I Relaxing continuous gives -18.5249
Cbc0038I Before mini branch and bound, 190 integers at bound fixed and 285 continuous
Cbc0038

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ciaran/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/ffc05451a30548f2b1f35684ffc1c3d6-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/ffc05451a30548f2b1f35684ffc1c3d6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 970 COLUMNS
At line 3470 RHS
At line 4436 BOUNDS
At line 4727 ENDATA
Problem MODEL has 965 rows, 482 columns and 1923 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 18.5249 - 0.00 seconds
Cgl0004I processed model has 573 rows, 475 columns (190 integer (190 of which binary)) and 1520 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -18.5249
Cbc0038I Relaxing continuous gives -18.5249
Cbc0038I Before mini branch and bound, 190 integers at bound fixed and 285 continuous
Cbc0038

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ciaran/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/bbbf721e4177430282bc394d44342a48-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/bbbf721e4177430282bc394d44342a48-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 970 COLUMNS
At line 3470 RHS
At line 4436 BOUNDS
At line 4727 ENDATA
Problem MODEL has 965 rows, 482 columns and 1923 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 18.5249 - 0.00 seconds
Cgl0004I processed model has 573 rows, 475 columns (190 integer (190 of which binary)) and 1520 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -18.5249
Cbc0038I Relaxing continuous gives -18.5249
Cbc0038I Before mini branch and bound, 190 integers at bound fixed and 285 continuous
Cbc0038

DAM-Total Profit for Alpha 0.5-0.5: 23882.363700000005

BM-Total Profit for Alpha 0.5-0.5: 18346.694499999994

Code for the DAM:

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus
from matplotlib.pyplot import figure

# Load DAM data
date_format_dam = "%d/%m/%Y %H:%M"
date_parse_dam = lambda date: dt.datetime.strptime(date, date_format_dam)
dat1 = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/DAM_QRA/QR/rf_Q_DAM_1-12.csv")
dat1 = pd.DataFrame(dat1)

quantiles = [10, 30, 50, 70, 90]
Q_DAM = {}
for q in quantiles:
    column_names = [f"EURPrices+{i}_Forecast_{q}" for i in range(24)]
    Q_DAM[q] = dat1[column_names].dropna().stack().reset_index()
    Q_DAM[q]["Price"] = Q_DAM[q].iloc[:, 2]
    Q_DAM[q] = Q_DAM[q].reindex(Q_DAM[q].index.repeat(2)).reset_index(drop=True)

column_names_dam = [f"EURPrices+{i}" for i in range(24)]
Y_r_DAM = dat1[column_names_dam].dropna().stack().reset_index()
Y_r_DAM = Y_r_DAM.reindex(Y_r_DAM.index.repeat(2)).reset_index(drop=True)
Y_r_DAM["Price"] = Y_r_DAM.iloc[:, 2]

# Set battery parameters
battery_efficiency_charge = 0.80
battery_efficiency_discharge = 0.98
Total_Daily_Volume = 6
max_battery_capacity = 1
min_battery_capacity = 0  
ramp_rate_charge = 1
ramp_rate_discharge = 1

# Define time periods (48-hour windows)
time_periods = range(0, len(Y_r_DAM), 48)

# Define quantile pairs
quantile_pairs = [(0.5, 0.5), (0.3, 0.7), (0.1, 0.9)]

# Initialize dictionaries to store total profits for each quantile pair
total_profits = {pair: 0 for pair in quantile_pairs}

# Initialize battery charge at the start of the first day
initial_charge = min_battery_capacity  

# Loop through each quantile pair
for alpha, complement_alpha in quantile_pairs:
    results = []
    current_charge = initial_charge

    for start_idx in time_periods:
        end_idx = start_idx + 48

        if end_idx > len(Y_r_DAM):
            break

        print(f"Initial charge: {current_charge}")

        alpha_key = int(alpha * 100)
        complement_alpha_key = int(complement_alpha * 100)

        if alpha_key not in Q_DAM or complement_alpha_key not in Q_DAM:
            continue

        p_max = Q_DAM[alpha_key]["Price"].iloc[start_idx:end_idx].values
        p_min = Q_DAM[complement_alpha_key]["Price"].iloc[start_idx:end_idx].values

        prob = LpProblem("Multiple_Trades_Quantile_Strategy", LpMaximize)
        buy_action = LpVariable.dicts("Buy_Action", range(48), cat='Binary')
        sell_action = LpVariable.dicts("Sell_Action", range(48), cat='Binary')
        buy = LpVariable.dicts("Buy", range(48), lowBound=0, cat='Continuous')
        sell = LpVariable.dicts("Sell", range(48), lowBound=0, cat='Continuous')
        charge = LpVariable.dicts("Charge", range(49), lowBound=min_battery_capacity, upBound=max_battery_capacity, cat='Continuous')

        # Initial battery charge constraint
        prob += charge[0] == current_charge, "Initial_Charge"

        # Define the total daily volume constraint
        prob += lpSum([buy[t] for t in range(48)]) + lpSum([sell[t] for t in range(48)]) == Total_Daily_Volume, "Total_Daily_Volume"

        # Ensure that each hour has at most one buy or sell action
        for t in range(48):
            prob += buy_action[t] + sell_action[t] <= 1, f"One_Action_Per_Hour_{t}"

        # Link action variables to buy and sell variables
        for t in range(48):
            prob += buy[t] <= ramp_rate_charge * buy_action[t], f"Buy_Action_Link_{t}"
            prob += sell[t] <= ramp_rate_discharge * sell_action[t], f"Sell_Action_Link_{t}"

        # Battery charge constraints
        for t in range(48):
            prob += charge[t + 1] == charge[t] + buy[t] - sell[t], f"Charge_Update_{t}"
            prob += charge[t] <= max_battery_capacity, f"Max_Capacity_{t}"
            prob += charge[t] >= min_battery_capacity, f"Min_Capacity_{t}"

            # Charging constraints
            prob += buy[t] <= max_battery_capacity - charge[t], f"Charging_Constraint_{t}"
            prob += buy[t] <= ramp_rate_charge, f"Charging_Ramp_Rate_Constraint_{t}"

            # Discharging constraints
            prob += sell[t] <= charge[t] - min_battery_capacity, f"Discharging_Constraint_{t}"
            prob += sell[t] <= ramp_rate_discharge, f"Discharging_Ramp_Rate_Constraint_{t}"

        # Define the objective function (maximize profit)
        prob += lpSum([(battery_efficiency_discharge * p_max[t] * sell[t]) - ((p_min[t] / battery_efficiency_charge) * buy[t]) for t in range(48)])

        # Solve the problem
        prob.solve()

        # Check if the solution is optimal
        if LpStatus[prob.status] == 'Optimal':
            # Extract buy and sell times and amounts
            buy_times = [t for t in range(48) if buy[t].varValue > 0]
            sell_times = [t for t in range(48) if sell[t].varValue > 0]

            # Ensure unique buy-sell pairs
            used_hours = set()
            valid_trades = []
            for bt in buy_times:
                if bt in used_hours:
                    continue
                for st in sell_times:
                    if st in used_hours or bt >= st:
                        continue

                    # Calculate expected profit
                    buy_price = p_min[bt]
                    sell_price = p_max[st]
                    expected_profit = (battery_efficiency_discharge * sell_price * sell[st].varValue) - ((buy_price * buy[bt].varValue) / battery_efficiency_charge)

                    # Only consider the trade if expected profit is greater than 0
                    if expected_profit > 0:
                        valid_trades.append((bt, st))
                        used_hours.add(bt)
                        used_hours.add(st)
                        break  # Only one valid sell per buy to ensure no repeated actions

            for bt, st in valid_trades:
                # Calculate profit using real prices
                buy_price = Y_r_DAM.iloc[start_idx + bt]["Price"]
                sell_price = Y_r_DAM.iloc[start_idx + st]["Price"]
                buy_amount = buy[bt].varValue
                sell_amount = sell[st].varValue
                profit = (battery_efficiency_discharge * sell_price * sell_amount) - ((buy_price * buy_amount) / battery_efficiency_charge)
                results.append(profit)
                print(f"Day {start_idx // 48 + 1}, Alpha {alpha}-{complement_alpha}: Buy at {bt} ({buy_amount} MW), Sell at {st} ({sell_amount} MW), Profit: {profit}")

            # Update the current charge for the next day
            current_charge = charge[48].varValue

    # Calculate the sum of profits for the current quantile pair
    total_profit = sum(results)
    total_profits[(alpha, complement_alpha)] = total_profit
    print(f"Total Profit for Alpha {alpha}-{complement_alpha}: {total_profit}\n")

# Print the total profits for all quantile pairs
for pair, profit in total_profits.items():
    print(f"Total Profit for Alpha {pair[0]}-{pair[1]}: {profit}")

Code for the BM:


In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus
from matplotlib.pyplot import figure

# Load data
date_format = "%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("/home/ciaran/QRA&Q-Ave(QR&CP)/BM_QRA/QR/rf_Q_1-12.csv")
dat1 = pd.DataFrame(dat)

# Create quantile dataframes
quantiles = [10, 30, 50, 70, 90]
Q = {}
for q in quantiles:
    column_names = [f"lag_{i}y_Forecast_{q}" for i in range(2, 18)]
    Q[q] = dat1[column_names].dropna().stack().reset_index()
    Q[q]["Price"] = Q[q].iloc[:, 2]

# Create a dataframe 'Y_r' with real price data
column_names = [f"lag_{i}y" for i in range(2, 18)]
Y_r = dat1[column_names].dropna().stack().reset_index()
Y_r["Price"] = Y_r.iloc[:, 2]

# Set battery parameters
battery_efficiency_charge = 0.80
battery_efficiency_discharge = 0.98
Total_Daily_Volume = 6
max_battery_capacity = 1
min_battery_capacity = 0
ramp_rate_charge = 1
ramp_rate_discharge = 1

# Define time periods (8-hour/16 periods windows)
time_periods = range(0, len(Y_r), 48)

# Define quantile pairs
quantile_pairs = [(0.5, 0.5)]

# Initialize dictionaries to store total profits for each quantile pair
total_profits = {pair: 0 for pair in quantile_pairs}

# Initialize battery charge at the start of the first day
initial_charge = min_battery_capacity

# Loop through each quantile pair
for alpha, complement_alpha in quantile_pairs:
    results = []
    current_charge = initial_charge

    for start_idx in time_periods:
        end_idx = start_idx + 48

        if end_idx > len(Y_r):
            break

        # Print initial charge for debugging
        print(f"Initial charge: {current_charge}")

        # Find the appropriate quantiles
        alpha_key = int(alpha * 100)
        complement_alpha_key = int(complement_alpha * 100)

        if alpha_key not in Q or complement_alpha_key not in Q:
            continue

        p_max = Q[alpha_key]["Price"].iloc[start_idx:end_idx].values
        p_min = Q[complement_alpha_key]["Price"].iloc[start_idx:end_idx].values

        # Define MILP problem
        prob = LpProblem("Multiple_Trades_Quantile_Strategy", LpMaximize)

        # Define decision variables
        buy_action = LpVariable.dicts("Buy_Action", range(48), cat='Binary')
        sell_action = LpVariable.dicts("Sell_Action", range(48), cat='Binary')
        buy = LpVariable.dicts("Buy", range(48), lowBound=0, cat='Continuous')
        sell = LpVariable.dicts("Sell", range(48), lowBound=0, cat='Continuous')
        charge = LpVariable.dicts("Charge", range(49), lowBound=min_battery_capacity, upBound=max_battery_capacity, cat='Continuous')

        # Initial battery charge constraint
        prob += charge[0] == current_charge, "Initial_Charge"

        # Define the total daily volume constraint
        prob += lpSum([buy[t] for t in range(48)]) <= Total_Daily_Volume, "Total_Daily_Volume_Buy"
        prob += lpSum([sell[t] for t in range(48)]) <= Total_Daily_Volume, "Total_Daily_Volume_Sell"

        # Ensure that each hour has at most one buy or sell action
        for t in range(48):
            prob += buy_action[t] + sell_action[t] <= 1, f"One_Action_Per_Hour_{t}"

        # Link action variables to buy and sell variables
        for t in range(48):
            prob += buy[t] <= ramp_rate_charge * buy_action[t], f"Buy_Action_Link_{t}"
            prob += sell[t] <= ramp_rate_discharge * sell_action[t], f"Sell_Action_Link_{t}"

        # Battery charge constraints
        for t in range(48):
            prob += charge[t + 1] == charge[t] + buy[t] - sell[t], f"Charge_Update_{t}"
            prob += charge[t] <= max_battery_capacity, f"Max_Capacity_{t}"
            prob += charge[t] >= min_battery_capacity, f"Min_Capacity_{t}"

            # Charging constraints
            prob += buy[t] <= max_battery_capacity - charge[t], f"Charging_Constraint_{t}"
            prob += buy[t] <= ramp_rate_charge, f"Charging_Ramp_Rate_Constraint_{t}"

            # Discharging constraints
            prob += sell[t] <= charge[t] - min_battery_capacity, f"Discharging_Constraint_{t}"
            prob += sell[t] <= ramp_rate_discharge, f"Discharging_Ramp_Rate_Constraint_{t}"
            
        # Add constraint for maximum difference between buy and sell timestamps
        for t_buy in range(48):
            for t_sell in range(48):
                if t_sell > t_buy:
                    prob += sell_action[t_sell] <= buy_action[t_buy] + (1 - buy_action[t_buy]) * (1 + (t_sell - t_buy) / 16), f"Max_Time_Diff_{t_buy}_{t_sell}"


        # Define the objective function (maximize profit)
        prob += lpSum([(battery_efficiency_discharge * p_max[t] * sell[t]) - ((p_min[t] / battery_efficiency_charge) * buy[t]) for t in range(48)])

        # Solve the problem
        prob.solve()

        # Check if the solution is optimal
        if LpStatus[prob.status] == 'Optimal':
            # Extract buy and sell times and amounts
            buy_times = [t for t in range(48) if buy[t].varValue > 0]
            sell_times = [t for t in range(48) if sell[t].varValue > 0]

            # Ensure unique buy-sell pairs
            used_hours = set()
            valid_trades = []
            for bt in buy_times:
                if bt in used_hours:
                    continue
                for st in sell_times:
                    if st in used_hours or bt >= st:
                        continue

                    # Calculate expected profit
                    buy_price = p_min[bt]
                    sell_price = p_max[st]
                    expected_profit = (battery_efficiency_discharge * sell_price * sell[st].varValue) - ((buy_price * buy[bt].varValue) / battery_efficiency_charge)

                    # Only consider the trade if expected profit is greater than 0
                    if expected_profit > 0:
                        valid_trades.append((bt, st))
                        used_hours.add(bt)
                        used_hours.add(st)
                        break  # Only one valid sell per buy to ensure no repeated actions

            for bt, st in valid_trades:
                # Calculate profit using real prices
                buy_price = Y_r.iloc[start_idx + bt]["Price"]
                sell_price = Y_r.iloc[start_idx + st]["Price"]
                buy_amount = buy[bt].varValue
                sell_amount = sell[st].varValue
                profit = (battery_efficiency_discharge * sell_price * sell_amount) - ((buy_price * buy_amount) / battery_efficiency_charge)
                results.append(profit)
                print(f"Day {start_idx // 48 + 1}, Alpha {alpha}-{complement_alpha}: Buy at {bt} ({buy_amount} MW), Sell at {st} ({sell_amount} MW), Profit: {profit}")

            # Update the current charge for the next day
            current_charge = charge[48].varValue

    # Calculate the sum of profits for the current quantile pair
    total_profit = sum(results)
    total_profits[(alpha, complement_alpha)] = total_profit
    print(f"Total Profit for Alpha {alpha}-{complement_alpha}: {total_profit}\n")

# Print the total profits for all quantile pairs
for pair, profit in total_profits.items():
    print(f"Total Profit for Alpha {pair[0]}-{pair[1]}: {profit}")


Total Profit for Alpha 0.5-0.5: 18346.694499999994